In [1]:
import time
import gym
import tensorflow as tf

In [2]:
env=gym.make('Alien-v0')
print(env.action_space)
print(env._max_episode_steps)

Discrete(18)
10000


In [ ]:
scores=[]
train_data=[]
accepted_score=[]
required_score=100

for i in range(100):
    if i %100==0:
        print(f'iter>{i}')
    env.reset()
    score=0
    game_memory=[]
    prev_obs=[]
    for j in range(1000):
        
        action=env.action_space.sample()
        obs,reward,done,info=env.step(action)
        
        if len(prev_obs)>0:
            game_memory.append([prev_obs,action])
        prev_obs=obs
        
        #env.render()
        #time.sleep(0.013)
        score+=reward
        if done:            
            break
    scores.append(score)
    if score>required_score:
        accepted_score.append(score)
        for data in game_memory:
            train_data.append(data)
print(score, step)

iter>0
